This walkthrough-style Jupyter notebook will detail the methodology used to evaluate the predictive performance of the underlying forecasting models in the Residential Energy Demand Forecasting Pipeline.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import pickle as pkl
import torch as t
from scipy.stats import wilcoxon
import plotly.graph_objects as go
import os
import warnings
import model_definitions as md
import preprocessing as pp
import numpy as np
import get_data as gd

Fit Forecasting Model on all training data

Load best fitting Forecaster object or fit a new one

In [50]:
load_model = True
model_path = r"Saved/Models/forecaster.pkl"

if load_model and os.path.exists(model_path): # load existing model
    with open(model_path, "rb") as file:
        model = pkl.load(file)

else: # fit new model - want to load these from repository
    best_pv_hyperparameters = {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.01, 'point_var_lags': 10, 'minimum_error_prediction': 100, 'error_trend': 100}
    best_lstm_hyperparameters = {"batch_size":100, "lr":0.0005, "dropout":0.1, "num_layers":1, "hidden_size":32, "sequence_length":24*7*3, "max_epochs":100}

    # load in clean dataset
    clean_data = pd.read_csv(r"Saved/Datasets/clean_training.csv", index_col=0)
    clean_data.index = pd.to_datetime(clean_data.index)
    clean_data.loc[clean_data["HourlyPrecipitation"].isna(),"HourlyPrecipitation"] = "None"
    # clean_data.loc[:,"HourlyPrecipitation"] = clean_data["HourlyPrecipitation"].replace({np.nan:"None"})

    # define model
    model = md.Forecaster(short_term_horizon=24)

    # fit model
    warnings.filterwarnings('ignore')
    model.fit(clean_training_data=clean_data, dependent_variable="Energy Demand (MWH)", 
        hyperparameters=best_pv_hyperparameters | best_lstm_hyperparameters, verbose=False)

<br>

# Validation

### Evaluating the Model on Holdout Data

With a final optimized forecasting model defined, it is important to evaluate its performance on unseen data and to contextualize that performance by comparing it with the performance of a baseline model. Below, we load the holdout dataset that was set aside during the processing phase of the pipeline and use it to evaluate the final forecasting model. The *Forecaster* class produces forecasts when the *predict* function is called and given a number, *hours_ahead*, of forecasts to predict. It returns both point forecasts and variance forecasts which can be used to create confidence intervals or relative confidence measures for each point forecasts. 

Below, we produce these forecasts and plot them alongside the actual values. The variance forecasts are used to calculate a 95% confidence interval (CI) for each point forecast.

### Evaluate Long-term Forecasting Performance

In [52]:
# load holdout data
evaluation_data = pd.read_csv(r"Saved/Datasets/holdout.csv", index_col=0)
evaluation_data.index = pd.to_datetime(evaluation_data.index)

# process holdout data
processor = pp.PreprocessingPipeline(save_datasets=False, produce_eda_plots=False)
evaluation_data = processor.process_dataset(evaluation_data) # HERE need to use previous Processor to process this dataset, use the same Prophet models to impute values
evaluation_data.loc[evaluation_data["HourlyPrecipitation"].isna(),"HourlyPrecipitation"] = "None"
display(evaluation_data["HourlyPrecipitation"].value_counts(dropna=False))

Interpolating for variable Energy Demand (MWH)
Interpolating for variable HourlyDryBulbTemperature
Interpolating for variable HourlyDewPointTemperature
Interpolating for variable HourlyWetBulbTemperature
Interpolating for variable HourlyStationPressure
Interpolating for variable HourlyWindSpeed
Interpolating for variable Energy Price (cents/KWH)
Interpolating for variable Civilian Noninstitutional Population
Interpolating for variable Labor Force Participation
Interpolating for variable CPI-U
Interpolating for variable HourlyPrecipitation


HourlyPrecipitation
None           4546
Light Rain      351
Medium Rain     190
Heavy Rain       15
Name: count, dtype: int64

In [36]:
display(evaluation_data[evaluation_data["HourlyPrecipitation"].isna()])

,Energy Demand (MWH),HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyWetBulbTemperature,HourlyStationPressure,HourlyPrecipitation,HourlyWindSpeed,Energy Price (cents/KWH),Civilian Noninstitutional Population,Labor Force Participation,CPI-U


In [11]:
# get complete range to be predicted (in case the holdout set is missing any values)
complete_range = pd.date_range(start=evaluation_data.index.min(), end=evaluation_data.index.max(), freq='H')

# produce long-term forecasts
lt_forecasts = model.long_term_predict(complete_range.shape[0])

# reconcile forecasts and ground truth values
results = pd.merge(lt_forecasts, evaluation_data, left_index=True, right_index=True, how="inner")
point_forecasts = results["Point Forecasts"].values
variance_forecasts = results["Variance Forecasts"].values
actual_values = results["Energy Demand (MWH)"]

display(lt_forecasts)

,Point Forecasts,Variance Forecasts
ds,,
2023-09-14 10:00:00,5262.630907,7810.351928
2023-09-14 11:00:00,5511.100358,14931.091105
2023-09-14 12:00:00,5842.862284,33657.582382
2023-09-14 13:00:00,6180.603293,67251.789910
2023-09-14 14:00:00,6452.496414,109893.751222
...,...,...
2024-04-13 20:00:00,4953.253668,732722.061877
2024-04-13 21:00:00,4948.897911,754190.917154
2024-04-13 22:00:00,4923.160970,749100.519916


In [59]:
# Combine error forecasts
std_forecasts = np.sqrt(variance_forecasts)

# 1.96 is the z-value for the 2.5th percentile of the standard normal distribution
lower_cuts = point_forecasts - 1.96 * std_forecasts  
upper_cuts = point_forecasts + 1.96 * std_forecasts

# Create figure showing forecasts and actual values
long_term_fig = go.Figure()
long_term_fig.add_trace(go.Scatter(x=evaluation_data.index, y=point_forecasts, mode='lines', 
    name='Point Forecasts', line=dict(color='rgba(255, 0, 0)')))
long_term_fig.add_trace(go.Scatter(x=evaluation_data.index, y=actual_values, mode='lines', 
    name='Ground Truth', line=dict(color='rgba(0, 0, 255)')))
long_term_fig.add_trace(go.Scatter(x=evaluation_data.index, y=upper_cuts, mode='lines', 
    name='Upper 95 CI', line=dict(color='rgba(0, 255, 0, 0.2)')))
long_term_fig.add_trace(go.Scatter(x=evaluation_data.index, y=lower_cuts, mode='lines', 
    name='Lower 95 CI', line=dict(color='rgba(200, 125, 200, 0.2)')))

# Customize the layout
long_term_fig.update_layout(
    title="Long-term Evaluation of Forecasting Pipeline with Holdout Data",
    xaxis_title='Time',
    yaxis_title='Energy Demand (MWH)',
    template='plotly_dark' 
)

# save plotly figure for web application
with open(r"assets/long_term_evaluation_plot.pkl", 'wb') as file:
    pkl.dump(long_term_fig, file=file)

long_term_fig.show()

In [13]:
### statistical comparison between baseline forecasting model and created forecasting model ###

# define baseline model as simple year-long moving average
sma = clean_data["Energy Demand (MWH)"].rolling(window=24*365).mean()
baseline_prediction = sma.iloc[-1]
sma_error = pd.merge(clean_data["Energy Demand (MWH)"], sma, left_index=True, right_index=True, 
    how="inner", suffixes=["_actual", "_baseline_prediction"])
sma_error = sma_error[sma_error["Energy Demand (MWH)_baseline_prediction"].notna()]
sma_variance = pd.Series((sma_error["Energy Demand (MWH)_baseline_prediction"] - 
    sma_error["Energy Demand (MWH)_actual"]).values**2).rolling(window=24*365).mean()
baseline_variance = sma_variance.iloc[-1]

# calculate errors from baseline model
baseline_errors = actual_values - baseline_prediction

# calculate errors from project model
model_errors = actual_values - point_forecasts

### calculate all measures for Forecasting Pipeline's long-term forecasts ###
# mse
pipeline_mse = np.sum((model_errors**2))/model_errors.shape[0]

# wmse
relative_confidence = 1/std_forecasts
relative_confidence = relative_confidence/np.sum(relative_confidence) * relative_confidence.shape[0]
pipeline_wmse = np.sum(relative_confidence * 
    (point_forecasts - actual_values)**2)/actual_values.shape[0]

# mae
pipeline_mae = np.sum(np.abs(model_errors))/model_errors.shape[0]

# mape
pipeline_mape = np.sum(np.abs(model_errors/actual_values))/model_errors.shape[0]

### calculate all measures for baseline model ###
# mse
baseline_mse = np.sum((baseline_errors**2))/baseline_errors.shape[0]
baseline_wmse = baseline_mse

# mae
baseline_mae = np.sum(np.abs(baseline_errors))/baseline_errors.shape[0]

# mape
baseline_mape = np.sum(np.abs(baseline_errors/actual_values))/baseline_errors.shape[0]

# calculate all measures for EIA forecasts

# compare MSE values
print("Baseline MSE: {:.2f}".format(baseline_mse))
print("Project Model MSE: {:.2f}".format(pipeline_mse))

# Perform Wilcoxon signed-rank test
statistic, p_value = wilcoxon(baseline_errors, model_errors)

# Output the test statistic and p-value
print("\nWilcoxon signed-rank test statistic:", statistic)
print("p-value: {:.4f}".format(p_value))

# Interpret the results
print("Outcome of Statistical Test:")
if p_value < 0.05:
    print("Reject the null hypothesis. There is a significant difference between the two models.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between the two models.")

# compare weighted MSE to regular MSE to see if confidence values help

print("\nProject Model Weighted MSE: {:.2f}".format(pipeline_wmse))

Baseline MSE: 600425.13
Project Model MSE: 138107.32

Wilcoxon signed-rank test statistic: 3461200.0
p-value: 0.0000
Outcome of Statistical Test:
Reject the null hypothesis. There is a significant difference between the two models.

Project Model Weighted MSE: 133002.95


In [14]:
# include MSE, wmse, Mape, weighted mape, statistical significance of each compared to baseline model
# create dataframe with results - easy to convert to html.
long_results_df = pd.DataFrame(index=["Forecasting Pipeline", "Baseline Yearly MA"], 
    data={"MSE":[pipeline_mse, baseline_mse], "Weighted MSE":[pipeline_wmse, baseline_wmse], 
    "MAE":[pipeline_mae, baseline_mae], "MAPE":[pipeline_mape, baseline_mape], "Wilcoxon Test p-value":[p_value, np.nan]}).round(2)
long_results_df.to_csv(r"assets/long_term_performance_report.csv")

### Evaluate Short-term Forecasting Performance

In [15]:
eia_forecast_data = gd.get_eia_forecasts(start=np.datetime64("2018-06-19"), end=np.datetime64("2024-04-15"))["EIA Forecast (MWH)"]

Requesting EIA energy demand forecast data from EIA over 11 requests


Obtain baseline model from EIA

In [16]:
eia_forecasts = eia_forecast_data[eia_forecast_data.index < evaluation_data.index.min()]
training_energy_demand = pd.read_csv(r"Saved/Datasets/preliminary.csv", index_col=0)["Energy Demand (MWH)"]
training_energy_demand.index = pd.to_datetime(training_energy_demand.index)
training_energy_demand = training_energy_demand[(training_energy_demand.index < evaluation_data.index.min())]
eia_forecasts = eia_forecasts[eia_forecasts.index.isin(training_energy_demand.index)]

coef = np.dot(eia_forecasts.values, training_energy_demand.values)/np.dot(eia_forecasts.values, eia_forecasts.values)
eia_forecasts = eia_forecast_data[eia_forecast_data.index.isin(evaluation_data.index)] * coef

In [53]:
sequence_length = 3*7*24
last_n_training = clean_data.iloc[-(sequence_length+model.short_term_horizon):]
evaluation_data_st = pd.concat([last_n_training, evaluation_data], axis=0)
evaluation_data_st.loc[evaluation_data_st["HourlyPrecipitation"].isna(),"HourlyPrecipitation"] = "None"

Calculate Forecasting Pipeline short-term forecasts

In [55]:
st_forecasts = model.short_term_predict(evaluation_data_st, expected_output_size=evaluation_data.shape[0]-model.short_term_horizon-sequence_length, 
    lstm_sequence_length=sequence_length)
display(st_forecasts)

,Point Forecast,Variance Forecast
2023-09-14 10:00:00,5291.629883,3607.024707
2023-09-14 11:00:00,5544.969238,3607.027326
2023-09-14 12:00:00,6005.209961,3606.922400
2023-09-14 13:00:00,6598.463867,3606.341893
2023-09-14 14:00:00,6989.659180,3606.344729
...,...,...
2024-04-13 20:00:00,5554.392578,3606.938804
2024-04-13 21:00:00,5567.265137,3607.611235
2024-04-13 22:00:00,5609.426270,3606.762453
2024-04-13 23:00:00,5643.481934,3606.175924


In [49]:
display(clean_data["HourlyPrecipitation"].value_counts(dropna=False))

HourlyPrecipitation
NaN            42336
Light Rain      2318
Medium Rain     1137
Heavy Rain       126
Name: count, dtype: int64

In [56]:
st_forecast_data = pd.concat([st_forecasts, eia_forecasts, actual_values], axis=1)
st_forecast_data = st_forecast_data[st_forecast_data.notna().all(axis=1)]
# st_forecast_data = st_forecast_data[st_forecast_data.index < np.datetime64("2024-01-01")]
display(st_forecast_data)
st_forecasts = st_forecast_data[["Point Forecast", "Variance Forecast"]]
eia_forecasts = st_forecast_data["EIA Forecast (MWH)"]
st_actual_values = st_forecast_data["Energy Demand (MWH)"]

,Point Forecast,Variance Forecast,EIA Forecast (MWH),Energy Demand (MWH)
2023-09-14 10:00:00,5291.629883,3607.024707,5002.735246,5282.0
2023-09-14 11:00:00,5544.969238,3607.027326,5421.531435,5679.0
2023-09-14 12:00:00,6005.209961,3606.922400,5622.875757,6040.0
2023-09-14 13:00:00,6598.463867,3606.341893,5624.553626,6338.0
2023-09-14 14:00:00,6989.659180,3606.344729,5564.485903,6531.0
...,...,...,...,...
2024-04-13 20:00:00,5554.392578,3606.938804,4864.814386,4849.0
2024-04-13 21:00:00,5567.265137,3607.611235,4927.231125,4878.0
2024-04-13 22:00:00,5609.426270,3606.762453,5045.017553,4916.0
2024-04-13 23:00:00,5643.481934,3606.175924,5139.313811,4935.0


In [58]:
# Create figure showing forecasts and actual values
short_term_fig = go.Figure()
short_term_fig.add_trace(go.Scatter(x=st_actual_values.index, y=eia_forecasts, mode='lines', 
    name='EIA Day-Ahead Forecasts', line=dict(color='rgba(255, 0, 0)')))
short_term_fig.add_trace(go.Scatter(x=st_actual_values.index, y=st_actual_values, mode='lines', 
    name='Ground Truth', line=dict(color='rgba(0, 0, 255)')))
short_term_fig.add_trace(go.Scatter(x=st_actual_values.index, y=st_forecasts["Point Forecast"], mode='lines', 
    name='Predicted Forecasts', line=dict(color='rgba(0, 0, 255)')))

# Customize the layout
short_term_fig.update_layout(
    title="Short-term Evaluation of Forecasting Pipeline with Holdout Data",
    xaxis_title='Time',
    yaxis_title='Energy Demand (MWH)',
    template='plotly_dark' 
)
short_term_fig.show()

with open(r"assets/short_term_evaluation_plot.pkl", "wb") as file:
    pkl.dump(short_term_fig, file)

In [152]:
### statistical comparison between Pipeline Forecasting Model and EIA (short-term forecasts) ###

# calculate errors from baseline model
eia_errors = st_actual_values - eia_forecasts

# calculate errors from project model
model_errors = st_actual_values - st_forecasts["Point Forecast"]

### calculate all measures for Forecasting Pipeline's short-term forecasts ###
# mse
pipeline_mse = np.sum((model_errors**2))/model_errors.shape[0]

# wmse
relative_confidence = 1/st_forecasts["Variance Forecast"].values
relative_confidence = relative_confidence/np.sum(relative_confidence) * relative_confidence.shape[0]
pipeline_wmse = np.sum(relative_confidence * 
    (st_forecasts["Point Forecast"] - st_actual_values)**2)/st_actual_values.shape[0]

# mae
pipeline_mae = np.sum(np.abs(model_errors))/model_errors.shape[0]

# mape
pipeline_mape = np.sum(np.abs(model_errors/st_actual_values))/model_errors.shape[0]

### calculate all measures for baseline model ###
# mse
baseline_mse = np.sum((eia_errors**2))/eia_errors.shape[0]
baseline_wmse = baseline_mse

# mae
baseline_mae = np.sum(np.abs(eia_errors))/eia_errors.shape[0]

# mape
baseline_mape = np.sum(np.abs(eia_errors/st_actual_values))/eia_errors.shape[0]

# calculate all measures for EIA forecasts

# compare MSE values
print("Baseline MSE: {:.2f}".format(baseline_mse))
print("Project Model MSE: {:.2f}".format(pipeline_mse))

# Perform Wilcoxon signed-rank test
statistic, p_value = wilcoxon(eia_errors, model_errors)

# Output the test statistic and p-value
print("\nWilcoxon signed-rank test statistic:", statistic)
print("p-value: {:.4f}".format(p_value))

# Interpret the results
print("Outcome of Statistical Test:")
if p_value < 0.05:
    print("Reject the null hypothesis. There is a significant difference between the two models.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between the two models.")

# compare weighted MSE to regular MSE to see if confidence values help

print("\nProject Model Weighted MSE: {:.2f}".format(pipeline_wmse))

Baseline MSE: 108554.60
Project Model MSE: 184321.98

Wilcoxon signed-rank test statistic: 1440524.0
p-value: 0.0000
Outcome of Statistical Test:
Reject the null hypothesis. There is a significant difference between the two models.

Project Model Weighted MSE: 184315.40


In [ ]:
### statistical comparison between baseline forecasting model and created forecasting model ###

# calculate errors from baseline model
eia_errors = st_actual_values - eia_forecasts

# calculate eia variance as daily moving average

# calculate errors from project model
# model_errors = actual_values - st_point_forecasts

# calculate all measures for forecasting pipeline
# mse
# pipeline_mse = np.sum((model_errors**2))/model_errors.shape[0]

# wmse
# relative_confidence = 1/std_forecasts
# relative_confidence = relative_confidence/np.sum(relative_confidence) * relative_confidence.shape[0]
# pipeline_wmse = np.sum(relative_confidence * 
#     (point_forecasts - actual_values)**2)/actual_values.shape[0]

# mae
# pipeline_mae = np.sum(np.abs(model_errors))/model_errors.shape[0]

# mape
# pipeline_mape = np.sum(np.abs(model_errors/actual_values))/model_errors.shape[0]

# calculate all measures for baseline model
# mse
eia_mse = np.sum((eia_errors**2))/eia_errors.shape[0]

# wmse

# mae
eia_mae = np.sum(np.abs(eia_errors))/eia_errors.shape[0]

# mape
eia_mape = np.sum(np.abs(eia_errors/st_actual_values))/eia_errors.shape[0]

# calculate all measures for EIA forecasts

# # compare MSE values
# print("Baseline MSE: {:.2f}".format(baseline_mse))
# print("Project Model MSE: {:.2f}".format(pipeline_mse))

# # Perform Wilcoxon signed-rank test
# statistic, p_value = wilcoxon(baseline_errors, model_errors)

# # Output the test statistic and p-value
# print("\nWilcoxon signed-rank test statistic:", statistic)
# print("p-value: {:.4f}".format(p_value))

# # Interpret the results
# print("Outcome of Statistical Test:")
# if p_value < 0.05:
#     print("Reject the null hypothesis. There is a significant difference between the two models.")
# else:
#     print("Fail to reject the null hypothesis. There is no significant difference between the two models.")

# # compare weighted MSE to regular MSE to see if confidence values help

# print("\nProject Model Weighted MSE: {:.2f}".format(pipeline_wmse))

In [ ]:
# include MSE, wmse, Mape, weighted mape, statistical significance of each compared to baseline model
# create dataframe with results - easy to convert to html.
day_ahead_results_df = pd.DataFrame(index=["Forecasting Pipeline", "EIA Forecasts"], 
    data={"MSE":["TBD", eia_mse], "Weighted MSE":["TBD", "TBD"], 
    "MAE":["TBD", eia_mae], "MAPE":["TBD", eia_mape], "Wilcoxon Test p-value":["TBD", np.nan]})
day_ahead_results_df = day_ahead_results_df.replace({"TBD":-888}).astype(float).round(2).replace({-888:"TBD"})
display(day_ahead_results_df)
day_ahead_results_df.to_csv(r"assets/short_term_performance_report.csv")

### Conduct Sensitivity Analysis

In [ ]:
# Sensitivity Analysis for day-ahead forecasts 
import seaborn as sns
sa_fig = plt.figure(figsize=(8,5))
ax = sa_fig.add_subplot()
sns.barplot(x=[0.85, 0.11, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05], y=clean_data.columns)
ax.bar_label(ax.containers[0])
ax.set_ylabel("Variable")
ax.set_xlabel("Feature Importance")
ax.set_title("Forecasting Model Feature Importance (Demonstration Only)")
ax.set_xlim([0,1])
plt.tight_layout()
pp.save_png_encoded(r'assets/sensitivity_analysis.png', sa_fig)

### Create Report for Download on the Web Application

In [ ]:
from plotly.io import to_html
with open(r"assets/model_performance_report.html", "w") as file:
    file.write("""
        <!DOCTYPE html>
        <html>
        <head>
        <title> Energy Demand Forecasting Model Performance Report </title>
        </head>
        <body>
        <H1> Energy Demand Forecasting Model Performance Report </H1>
        <div style="width: 75%;">The following report contains model performance metrics for the NY City Hourly Probabilistic Residential Energy Demand Forecasting Pipeline. 
            Model performance was evaluated on both long-term and day-ahead forecasts. Evaluation was conducted using a holdout dataset of hourly energy 
            demand values between {} and {}.
        <br>
        <H3> Long-Term Forecasting Performance </H3>
        <div style="width: 75%;"> The following table contains performance metrics for the forecasting model compared with a yearly moving average baseline model.</div>
        <div style="width: 75%;">{}</div>
        <br>
        <div style="width: 75%;"> The following Plotly Figure helps to contextualize the forecasting model's performance by showing its predictions along with the actual energy demand 
            values. It also presents the 95% confidence interval bounds estimated by the forecasting model.</div>
        <div style="width: 75%;">{}</div>
        <br> <br> <br>  
        <H3> Day-Ahead Forecasting Performance </H3>
        <div style="width: 75%;"> The following table contains performance metrics for the forecasting model compared with a yearly moving average baseline model.</div>
        <div style="width: 75%;">{}</div>
        <br>
        <divs style="width: 75%;"> The following Plotly Figure helps to contextualize the forecasting model's performance by showing its predictions along with the actual energy demand 
            values. It also presents the 95% confidence interval bounds estimated by the forecasting model. </div>
        <div style="width: 75%;">{}</div>
        <br> <br> <br>
        <H3>Model Sensitivity Analysis</H3>
        <div style="width: 75%;">The following data has been made up for demonstration purposes. Actual data will be provided in a future update.</div>
        <br>
        <img src="{}" width="50%" alt="Sensitivity Analysis">
        </body>
        """.format(evaluation_data.index.min().strftime("%Y-%m-%d"), evaluation_data.index.max().strftime("%Y-%m-%d"), long_results_df.to_html(), 
            to_html(long_term_fig), day_ahead_results_df.to_html(), to_html(short_term_fig), r"sensitivity_analysis.png"))

## Produce Final Model and Predict into the Future

### Fit Model on All Available Data

In [ ]:
# create new processing pipeline to process all data but not create visuals
processor = pp.PreprocessingPipeline(save_datasets=False, produce_eda_plots=False)
final_train_data,_ = processor.process_dataset(preliminary_dataset=preliminary_data, split_into_train_holdout=False)

In [ ]:
# fit final model on all available data
final_model = md.Forecaster()
final_model.fit(clean_training_data=final_train_data, dependent_variable="Energy Demand (MWH)", 
    strong_predictors=["HourlyDryBulbTemperature"], hyperparameters=best_set)

In [ ]:
# Forecast 2 years into the future
future_point_forecasts, future_variance_forecasts = final_model.predict(hours_ahead=365*24*2)

In [ ]:
# produce plot showing 2 year forecast
# Combine error forecasts
std_forecasts = np.sqrt(future_variance_forecasts)

# 1.96 is the z-value for the 2.5th percentile of the standard normal distribution
lower_cuts = future_point_forecasts - 1.96 * std_forecasts  
upper_cuts = future_point_forecasts + 1.96 * std_forecasts

# Create figure showing forecasts and actual values
fig = go.Figure()
fig.add_trace(go.Scatter(x=future_point_forecasts.index, y=future_point_forecasts, mode='lines', 
    name='Point Forecasts', line=dict(color='rgba(255, 0, 0)')))
fig.add_trace(go.Scatter(x=future_point_forecasts.index, y=upper_cuts, mode='lines', 
    name='Upper 95 CI', line=dict(color='rgba(0, 255, 0, 0.2)')))
fig.add_trace(go.Scatter(x=future_point_forecasts.index, y=lower_cuts, mode='lines', 
    name='Lower 95 CI', line=dict(color='rgba(200, 125, 200, 0.2)')))

# Customize the layout
fig.update_layout(
    title="Evaluating Forecasting Pipeline with Holdout Data",
    xaxis_title='Time',
    yaxis_title='Energy Demand (MWH)',
    template='plotly_dark',
    xaxis=dict(range=[future_point_forecasts.index[0],future_point_forecasts.index[-1]])
)

In [ ]:
# save plotly figure for web app
with open(r"Saved/Plotly Figures/Forecasting/future_forecasts.pkl", 'wb') as file:
    pkl.dump(fig, file=file)

# save future forecasts as csv for web app
future_forecasts = pd.concat([future_point_forecasts.rename("Point Forecast"), future_variance_forecasts.rename("Variance Forecast")], 
    axis=1).rename_axis("Time").reset_index()
future_forecasts.to_csv("assets/future_forecasts.csv")